# Examples using SciKit-image for Al-silicate phase diagram 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from thermoengine import model,phases
%matplotlib inline

## Instantiate phases ...

In [ ]:
database = model.Database()
And = database.get_phase('And')
Sil = database.get_phase('Sil')
Ky = database.get_phase('Ky')

## Calculate all reactions and plot boundaries ...

In [ ]:
tmin =  200 # °C
tmax = 1000
pmin =  100 # bars
pmax = 7000 # bars
x, y = np.ogrid[tmin+273.15:tmax+273.15:20j, pmin:pmax:20j]
fig, ax = plt.subplots()

for react in range(0,3):
    mat = []
    for yy in y[0]:
        row = []
        for xx in x:
            if react == 0:
                row.append(And.gibbs_energy(xx[0],yy) - Sil.gibbs_energy(xx[0],yy))
            elif react == 1:
                row.append(And.gibbs_energy(xx[0],yy) - Ky.gibbs_energy(xx[0],yy))
            elif react == 2:
                row.append(Ky.gibbs_energy(xx[0],yy) - Sil.gibbs_energy(xx[0],yy))
            else:
                pass
        mat.append(row)
    r = np.array(mat)
    contours = measure.find_contours(r, 0.0)

    for n, contour in enumerate(contours):
        ax.plot(contour[:, 1]*(tmax-tmin)/20+tmin, contour[:, 0]*(pmax-pmin)/20+pmin, linewidth=2)

plt.xlabel('T °C')
plt.ylabel('P bars')
plt.show()

## Calculate energy fields for all phases ...

In [ ]:
tmin =  200 # °C
tmax = 1000
pmin =  100 # bars
pmax = 7000 # bars
nGrid = 100j
x, y = np.ogrid[tmin+273.15:tmax+273.15:nGrid, pmin:pmax:nGrid]

phases = []
for react in range(0,3):
    mat = []
    for yy in y[0]:
        row = []
        for xx in x:
            if react == 0:
                row.append(And.gibbs_energy(xx[0],yy))
            elif react == 1:
                row.append(Ky.gibbs_energy(xx[0],yy))
            elif react == 2:
                row.append(Sil.gibbs_energy(xx[0],yy))
            else:
                pass
        mat.append(row)
    phases.append(np.array(mat))

## Determine a minimal energy surface (or convex hull) ...

In [ ]:
gSection = []
for i in range(0,3):
    mask = np.full(phases[i].shape, True, dtype=bool)
    for j in range(0,3):
        if i == j:
            pass
        else:
            mask = np.logical_and(mask, np.less_equal(phases[i],phases[j],where=True))
    gSection.append(phases[i])
    gSection[i][np.invert(mask)] = 0

## Combine the parts ...

In [ ]:
gSurf = np.copy(gSection[0])
for i in range(1,3):
    gSurf += gSection[i]

## Plot the bounaries ...

In [ ]:
fig, ax = plt.subplots()
for i in range(0,3):
    contours = measure.find_contours(gSurf-phases[i], -0.1)
    for n, contour in enumerate(contours):
        ax.plot(contour[:, 1]*(tmax-tmin)/int(nGrid.imag)+tmin, contour[:, 0]*(pmax-pmin)/int(nGrid.imag)+pmin, linewidth=2)
plt.xlabel('T °C')
plt.ylabel('P bars')
plt.show()